In [220]:
import numpy as np
from hmm_utils import *
from itertools import product

In [221]:
def compute_1st_part_y_t(z_matrix, rho_matrix, n_observations):
    n_observations = z_matrix.shape[1]
    j_obs = np.arange(0,n_observations)
    return np.dot(z_matrix* rho_matrix, j_obs)

def compute_2nd_part_y_t(z_matrix, rho_matrix, x_vector):
    return np.sum(z_matrix *(1-rho_matrix)* x_vector, axis = 1)

def compute_y_t(z_matrix, rho_matrix, x_vector):
    t, n_obs = z_matrix.shape
    y_t_1 = compute_1st_part_y_t(z_matrix, rho_matrix, n_obs)
    y_t_2 = compute_2nd_part_y_t(z_matrix, rho_matrix, x_vector)
    y_t = (y_t_1 + y_t_2).reshape(t,1)
    return y_t

def state_attraction_repulsion_f1(alpha_t,beta_t,c,state):
    numerator = alpha_t + beta_t
    denominator = np.sum(numerator ,axis=0)
    result = c * (numerator - denominator)
    return result[:,state]

def f2_function(z_matrix, x_obs_vector):
    t = z_matrix.shape[0]
    z_vector = np.where(z_matrix == 1)[1].reshape(t,1)
    vec_diff = z_vector - x_obs_vector
    return np.count_nonzero(vec_diff)

def utility_u1_state_attraction_repulsion_function(hmm, c, state, w1, w2, z_matrix, x_obs_vector):
    alpha = hmm.alpha(x_obs_vector)
    beta = hmm.beta(x_obs_vector)
    u_part_1 = w1 * state_attraction_repulsion_f1(alpha, beta, c,state)
    u_part_2 = w2 * f2_function(z_matrix, x_obs_vector)
    return u_part_1 + u_part_2


def generate_rho_sample(t, n_obs, theta_v = None, params_beta = {'a':1,'b':1}):
    
    if theta_v == None:
        params_beta['size'] = n_obs
        theta_v = np.random.beta(**params_beta)
    list_sample = []    
    for n in range(n_obs):
        list_sample.append(np.random.choice([0,1],p=[theta_v[n],1-theta_v[n]],size=t))

    return np.array(list_sample).T

def generate_Z_set(n_obs,t):
    diag_matrix =  np.diag(np.ones(n_obs))
    z_elements  = np.split(diag_matrix, n_obs)
    return list(product((z_elements), repeat = t))

In [222]:
priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                     [1/2, 1/4, 1/4]])
emission   = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
                    [0.1, 0.1, 0.1, 0.1, 0.1, 0.5],
                       [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]])

m = HMM(3)

m.startprob_ = priors
m.transmat_ = transition
m.emissionprob_ = emission

In [223]:
Z_M = np.array([[1, 0, 0],
                [0, 1, 0],
                [1, 0, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0]])

rho_M = np.array([[1,1,1],
                 [1,1,1],
                 [0,1,1],
                 [1,1,1],
                 [1,1,1],
                 [1,1 ,1]])


X_M = np.array([[2],
               [2],
               [2],
               [2],
               [2],
               [2]])

In [224]:
utility_u1_state_attraction_repulsion_function(hmm = m, c = 1, state = 2, w1 = 1,w2 =2,z_matrix= Z_M,x_obs_vector =  X_M)

array([110.84240273, 108.24096227, 106.19843397, 104.26908223,
       102.3589218 , 100.45406608])

In [225]:
y_t = compute_y_t(z_matrix = Z_M, rho_matrix =rho_M, x_vector = X_M)
utility_u1_state_attraction_repulsion_function(hmm = m, c = 1, state = 2, w1 = 1,w2 =2,z_matrix= Z_M,x_obs_vector =  y_t)

array([102.84240273, 100.24096227,  98.19843397,  96.26908223,
        94.3589218 ,  92.45406608])

In [226]:
n_obs = 5
t = 5
state = 1
hmm_model = m
c = 1
n = 1
Z_set = generate_Z_set(n_obs,t)
x_vector = m.sample(t)[0]
rho_matrix = generate_rho_sample(t, n_obs, theta_v = None, params_beta = {'a':1,'b':1})
u_dict = {}
for i,z in enumerate(Z_set):
    z_matrix = np.array(z).reshape(t,n_obs)
    y_t = compute_y_t(z_matrix, rho_matrix, x_vector).astype('int')
    u_value = utility_u1_state_attraction_repulsion_function(hmm_model, c, state, 1, 2, z_matrix, y_t)
    u_dict[i] = {'z':z_matrix, 'u':u_value}